In [ ]:
import glob
import ipywidgets
import os

### List Currently Installed Conda Environments

In [ ]:
%conda env list

### Select a Conda Environment to Create

Note: new environment.yml files should be stored in /home/jovyan/Environment_Configs/ and follow the naming convention \<environment_name\>_env.yml where \<environment_name\> is the environment name

In [ ]:
env_ymls = glob.glob("/home/jovyan/Environment_Configs/*.yml")
height = len(env_ymls) * 19
names =  ipywidgets.RadioButtons(
    options=[ os.path.basename(f).split('_env')[0] for f in env_ymls ],
    description="",
    disabled=False,
    layout=ipywidgets.Layout(height=f"{height}px", width='500px')
)
names

### Collect the Environment Name, Prefix, and Path to its env.yml

In [ ]:
yml = ""
name = names.value
for pth in env_ymls:
    if name in pth:
        yml = pth
        break
prefix = f"/home/jovyan/.local/envs/{name}"
print(yml)

### Create the Environment

In [ ]:
%conda env create -f $yml --prefix $prefix --force

### Update iykernel with the environment name and location of its kernel

In [ ]:
kernel_cmd = f"{prefix}/bin/python -m ipykernel install --user --name {name}"
print(kernel_cmd)

In [ ]:
!$kernel_cmd

In [ ]:
l_name = name.lower()
%conda run -n $name kernda /home/jovyan/.local/share/jupyter/kernels/$l_name/kernel.json --env-dir $prefix -o

### Restart Your Server Before Running any Installation Notebooks Associated with the New Environment or Notebooks That Use it.